# Importing required libraries


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split


# Defining the Paths to the Embeddings

In [ ]:
real_embeddings_path = '/kaggle/input/celebdfembedding/real/'
fake_embeddings_path = '/kaggle/input/celebdfembedding/fake/'


# Creating Custom Dataset Class

In [ ]:
class EmbeddingsDataset(Dataset):
    def __init__(self, real_embeddings_path, fake_embeddings_path):
        self.real_files = [os.path.join(real_embeddings_path, f) for f in os.listdir(real_embeddings_path) if f.endswith('.pt')]
        self.fake_files = [os.path.join(fake_embeddings_path, f) for f in os.listdir(fake_embeddings_path) if f.endswith('.pt')]
        self.files = self.real_files + self.fake_files
        self.labels = [1] * len(self.real_files) + [0] * len(self.fake_files)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = self.files[idx]
        label = self.labels[idx]
        embedding = torch.load(file_path, map_location=torch.device('cpu')).squeeze(0).view(-1)  #to flatten the embedding
        return embedding, torch.tensor(label, dtype=torch.float32)


# Loading the Dataset and Splitting into Training and Testing Sets

In [ ]:
dataset = EmbeddingsDataset(real_embeddings_path, fake_embeddings_path)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Defining the Neural Network Classifier

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x


# Initializing the Model, Loss Function, and Optimizer

In [ ]:
input_dim = 3137 * 768  #since we want to flatten the last two dimensions
model = SimpleNN(input_dim)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Training Loop

In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y.unsqueeze(1))  # Reshape batch_y to match output shape
        loss.backward()  # Backward pass and optimize
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')


# Evaluate the Model


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        outputs = model(batch_x)
        predicted = (outputs > 0.5).float()
        total += batch_y.size(0)
        correct += (predicted == batch_y.unsqueeze(1)).sum().item()  

accuracy = correct / total
print(f'Accuracy: {accuracy * 100:.2f}%')
